In [ ]:
#-----Loading in packages-----#
import scvi
import anndata as ad #building anndata objects
import scanpy as sc # Single cell analysis 
import pandas as pd # dataframe manipulation 
import numpy as np
from scipy.io import mmread #Reading in mtx file 
from scipy.sparse import csr_matrix #convert from coo to csr matrix
from rich import print
import seaborn as sns
import os

In [ ]:
#-----Setting Seed-----#
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)

In [ ]:
#------Generating Anndata from Seurat Object-----#
conversion_dir = "/projects/b1217/HHA/Multiome_Scanpy_Conversion/Full_Atlas/"
#Contains metadata, barcodes, and umap embeddings 
metadata = pd.read_csv(os.path.join(conversion_dir, "SCRNA_Multiome_IFE_Metadata_4_1_25.csv"))
#Reading in the count matrix from seurat.combined
counts = mmread(os.path.join(conversion_dir, "SCRNA_Multiome_IFE_RawCounts_4_1_25.mtx"))
counts = csr_matrix(counts.transpose()) #transposing to cell x gene matrix and converting to csr format
#Contains gene names 
gene_names = pd.read_csv(os.path.join(conversion_dir, "SCRNA_Multiome_IFE_GeneNames_4_1_25.csv"))
#Log normalized expression for stress genes
stress_genes = pd.read_csv(os.path.join(conversion_dir, "SCRNA_Multiome_IFE_StressSig_LogNorm_4_1_25.csv"))
stress_genes = stress_genes.set_index("Barcode")
#Log normalized expression for cell cycle genes
cc_genes = pd.read_csv(os.path.join(conversion_dir, "SCRNA_Multiome_IFE_CellCycle_LogNorm_4_1_25.csv"))
cc_genes = cc_genes.set_index("Barcode")
#Converting to anndata object
HHA = ad.AnnData(counts) #creating anndata object
HHA.obs_names = metadata["barcode"] #Adding barcodes as obs names 
HHA.var_names = gene_names["gene"] #adding gene names 
#Saving Count Layer
HHA.layers["counts"] = HHA.X.copy()
#Viewing results
print(HHA)

In [ ]:
#------Adding metadata to anndata--------#
HHA.obs = metadata
HHA.obs_names = metadata["barcode"]
#Adding normalized cell cycle gene expression to obs
HHA.obs = pd.concat([HHA.obs, cc_genes], axis =1)
#Adding normalized stress gene expression to obs
HHA.obs = pd.concat([HHA.obs, stress_genes], axis =1)
HHA.obs

In [ ]:
#-----Setting Up SCVI Model-----#
#Specifying the donor and sequencing platform as additional categorical covariates for integration
#Not including mitochondrial and ribosomal percentage or cell cycle as covariates, as this may mask real biology
continuous_covariates = stress_genes.columns.to_list() + cc_genes.columns.to_list() + ["nCount_RNA", "nFeature_RNA", "percent.mt", "percent.ribo"]
scvi.model.SCVI.setup_anndata(HHA, layer="counts", 
                              batch_key="SampleID",
                              categorical_covariate_keys = ["DonorID", "Sex", "Platform"],
                              continuous_covariate_keys = continuous_covariates)
model = scvi.model.SCVI(HHA, n_layers=2, n_latent=50, gene_likelihood= 'nb')
# view anndata_setup as a sanity check
model.view_anndata_setup()

In [ ]:
#Training model using gpu acceleration
model.train(max_epochs = 500, early_stopping = True) 

In [ ]:
#-----Plotting Model convergence-----#
train_elbo = model.history["elbo_train"][1:]
test_elbo = model.history["elbo_validation"]
ax = train_elbo.plot()
test_elbo.plot(ax=ax)

In [ ]:
#-----Extracting Latent Representation from Model----#
HHA.obsm["X_scVI"] = model.get_latent_representation()
HHA.obsm["X_scVI"]

#----Leiden Clustering with default Scanpy workflow------#
sc.pp.neighbors(HHA, use_rep= "X_scVI")
sc.tl.leiden(HHA)
sc.tl.umap(HHA)

In [ ]:
#----- Quickly Visualizing results with UMAP embeddding------#
sc.pl.embedding(
    HHA,
    basis="X_umap",
    color=['GlobalAnnotation', "SampleID", 'Sex', "Phase", 'leiden'],
    frameon=False,
    add_outline=True,
    ncols=1)

In [ ]:
#-----Checking QC Metric mixing in UMAP embeddings------#
sc.pl.embedding(HHA, basis="X_umap",
    color=["nCount_RNA", "nFeature_RNA", "percent.mt", "percent.ribo"],
    cmap = "plasma", frameon=False, add_outline=True, size = 7.5, ncols=2)
sc.pl.embedding(HHA, basis="X_umap",
    color=["DissociationScore"],
    cmap = "turbo", add_outline=True, size = 7.5, frameon=False, ncols=2)

In [ ]:
#-------Extracting SCVI Latent Dimensions and Writing to CSV-----#
HHA_Latent = pd.DataFrame(HHA.obsm["X_scVI"])
HHA_Latent.columns = ["SCVI_" + str(i) for i in range(1, 51)]
HHA_Latent["Barcode"] = HHA.obs_names
HHA_Latent.head(10)
HHA_Latent.to_csv(os.path.join(conversion_dir, "HHA_SCRNA_Multiome_RNA_IFE_LatentRep_4_1_25.csv"),
                               index = False)

In [ ]:
#----Rerunning model after filtering Fos/Jun outliers-----#
filtered_barcodes = pd.read_csv(os.path.join(conversion_dir, "HHA_IFE_Stress_Filtered_Barcodes_4_1_25.csv"))
HHA_Filtered = HHA[HHA.obs["barcode"].isin(filtered_barcodes["Barcode"])].copy()

In [ ]:
#-----Setting Up SCVI Model-----#
#Specifying the donor and sequencing platform as additional categorical covariates for integration
#Not including mitochondrial and ribosomal percentage or cell cycle as covariates, as this may mask real biology
continuous_covariates = stress_genes.columns.to_list() + cc_genes.columns.to_list() + ["nCount_RNA", "nFeature_RNA", "percent.mt", "percent.ribo"]
scvi.model.SCVI.setup_anndata(HHA_Filtered, layer="counts", 
                              batch_key="SampleID",
                              categorical_covariate_keys = ["DonorID", "Sex", "Platform"],
                              continuous_covariate_keys = continuous_covariates)
filtered_model = scvi.model.SCVI(HHA_Filtered, n_layers=2, n_latent=50, gene_likelihood= 'nb')
# view anndata_setup as a sanity check
filtered_model.view_anndata_setup()

In [ ]:
#Training model using gpu acceleration
filtered_model.train(max_epochs = 500, early_stopping = True) 

In [ ]:
#-----Plotting Model convergence-----#
train_elbo_filtered = filtered_model.history["elbo_train"][1:]
test_elbo_filtered = filtered_model.history["elbo_validation"]
ax = train_elbo_filtered.plot()
test_elbo_filtered.plot(ax=ax)

In [ ]:
#-----Extracting Latent Representation from Model----#
HHA_Filtered.obsm["X_scVI"] = filtered_model.get_latent_representation()
HHA_Filtered.obsm["X_scVI"]

#----Leiden Clustering with default Scanpy workflow------#
sc.pp.neighbors(HHA_Filtered, use_rep= "X_scVI")
sc.tl.leiden(HHA_Filtered)
sc.tl.umap(HHA_Filtered)

In [ ]:
#----- Quickly Visualizing results with UMAP embeddding------#
sc.pl.embedding(
    HHA_Filtered,
    basis="X_umap",
    color=['GlobalAnnotation', "SampleID", 'Sex', "Phase", 'leiden'],
    frameon=False,
    add_outline=True,
    ncols=1)

In [ ]:
#-----Checking QC Metric mixing in UMAP embeddings------#
sc.pl.embedding(HHA_Filtered, basis="X_umap",
    color=["nCount_RNA", "nFeature_RNA", "percent.mt", "percent.ribo"],
    cmap = "plasma", frameon=False, add_outline=True, size = 7.5, ncols=2)
sc.pl.embedding(HHA_Filtered, basis="X_umap",
    color=["DissociationScore"],
    cmap = "turbo", add_outline=True, size = 7.5, frameon=False, ncols=2)

In [ ]:
#------Checking on gene expression-----#
#Normalizing genes
sc.pp.normalize_total(HHA_Filtered)
sc.pp.log1p(HHA_Filtered)
#Plotting on UMAP
sc.pl.embedding(HHA_Filtered, basis="X_umap",
    color=["COL17A1", "KRT5", "KRT10", "KRT1", "KRTDAP", "IVL", "GATA3", "SOX6", "FOS"],
                add_outline=True, size = 15,
    cmap = "plasma", frameon=False, ncols=3)

In [ ]:
#-------Extracting SCVI Latent Dimensions and Writing to CSV-----#
HHA_Latent_Filtered = pd.DataFrame(HHA_Filtered.obsm["X_scVI"])
HHA_Latent_Filtered.columns = ["SCVI_" + str(i) for i in range(1, 51)]
HHA_Latent_Filtered["Barcode"] = HHA_Filtered.obs_names
HHA_Latent_Filtered.head(10)
HHA_Latent_Filtered.to_csv(os.path.join(conversion_dir, "HHA_SCRNA_Multiome_RNA_IFE_PostFiltering_LatentRep_4_1_25.csv"),
                               index = False)